# Lab 8 - Meotdy Sztucznej Inteligencji
## Autor: Dominik Pabiniak, 254397

##### Zaimplementuj Logic Gate Network, czyli sieć warstwową, w której każdy "neuron" wykorzystuje jeden z operatorów logicznych na wybranych wejściach. W trakcie uczenia, możliwa jest modyfikacja wyboru wejść i zmiana operatora. Wygeneruj zbiór danych na zainicjalizowanym modelu, a następnie spróbuj nauczyć inny model, wykorzystując np. algorytm genetyczny.

In [2]:
import numpy as np
import random

def AND(x1, x2):
    return x1 and x2

def OR(x1, x2):
    return x1 or x2

def NOT(x):
    return not x

def XOR(x1, x2):
    return x1 != x2

class LogicNeuron:
    def __init__(self, gate_type, input_indices):
        self.gate_type = gate_type
        self.input_indices = input_indices

    def activate(self, inputs):
        selected_inputs = [inputs[i] for i in self.input_indices]
        if self.gate_type == 'AND':
            return AND(*selected_inputs)
        elif self.gate_type == 'OR':
            return OR(*selected_inputs)
        elif self.gate_type == 'NOT':
            return NOT(*selected_inputs)
        elif self.gate_type == 'XOR':
            return XOR(*selected_inputs)
        else:
            raise ValueError("Unknown gate type")

class LogicGateNetwork:
    def __init__(self, layers):
        self.layers = layers

    def predict(self, inputs):
        for layer in self.layers:
            new_inputs = []
            for neuron in layer:
                new_inputs.append(neuron.activate(inputs))
            inputs = new_inputs
        return inputs[0]

In [10]:
neuron1 = LogicNeuron('AND', [0, 1])
neuron2 = LogicNeuron('OR', [0, 1])
neuron3 = LogicNeuron('NOT', [0])
neuron4 = LogicNeuron('XOR', [0, 1])

layer1 = [neuron1, neuron2]
layer2 = [neuron3, neuron4]
layer3 = [LogicNeuron('AND', [0, 1])]

model = LogicGateNetwork([layer1, layer2, layer3])

def generate_data(n):
    x = np.arange(0.0, n, 0.1)
    x, y = np.meshgrid(x, x)

    dataX = x.flatten()
    dataY = y.flatten()
    dataXY = np.column_stack((dataX, dataY))

    data_labels = []
    for pair in dataXY:
        x_val, y_val = pair
        data_labels.append(np.round(x_val) * np.round(y_val))

    return dataXY, np.array(data_labels)

n = 2
dataXY, data_labels = generate_data(n)

data = []

for i in range(len(dataXY)):
    x1, x2 = dataXY[i]
    output = data_labels[i]
    data.append(([x1, x2], output))

print("Generated Data:")
for entry in data:
    print(entry)
    break


Generated Data:
([0.0, 0.0], 0.0)


In [35]:
def fitness_function(network, data):
    correct = 0
    for inputs, output in data:
        if network.predict(inputs) == output:
            correct += 1
    return correct / len(data)

def random_neuron(input_size):
    gate_type = random.choice(['AND', 'OR', 'NOT', 'XOR'])
    if gate_type == 'NOT':
        input_indices = [random.randint(0, input_size - 1)]
    else:
        input_indices = random.sample(range(input_size), 2)
    return LogicNeuron(gate_type, input_indices)

def random_network(input_size, layer_sizes):
    layers = []
    for size in layer_sizes:
        layer = [random_neuron(input_size) for _ in range(size)]
        layers.append(layer)
    return LogicGateNetwork(layers)

def crossover(network1, network2):
    new_layers = []
    for layer1, layer2 in zip(network1.layers, network2.layers):
        new_layer = []
        for neuron1, neuron2 in zip(layer1, layer2):
            if random.random() < 0.5:
                new_layer.append(neuron1)
            else:
                new_layer.append(neuron2)
        new_layers.append(new_layer)
    return LogicGateNetwork(new_layers)

def mutate(network, mutation_rate):
    for i, layer in enumerate(network.layers):
        for neuron in layer:
            if random.random() < mutation_rate:
                neuron.gate_type = random.choice(['AND', 'OR', 'NOT', 'XOR'])
                
                previous_layer_size = len(network.layers[i - 1]) if i > 0 else len(layer)

                if neuron.gate_type == 'NOT':
                    neuron.input_indices = [random.randint(0, previous_layer_size - 1)]
                else:
                    if previous_layer_size > 1:
                        neuron.input_indices = random.sample(range(previous_layer_size), 2)
                    else:
                        neuron.input_indices = [0]
    return network


def genetic_algorithm(data, input_size, layer_sizes, population_size, generations, mutation_rate):
    population = [random_network(input_size, layer_sizes) for _ in range(population_size)]
    best_accuracy = 0
    
    for _ in range(generations):
        population.sort(key=lambda x: fitness_function(x, data), reverse=True)
        best_network = population[0]
        
        acc = fitness_function(best_network, data)
        if acc > best_accuracy:
            best_network_acc = best_network
            best_accuracy = acc
        if fitness_function(best_network, data) >= 0.7:
            break
        
        new_population = [best_network]
        while len(new_population) < population_size:
            parent1 = random.choice(population[:10])
            parent2 = random.choice(population[:10])
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)
        
        population = new_population
    return best_network_acc

input_size = 2
layer_sizes = [2, 5, 1]
population_size = 200
generations = 50
mutation_rate = 0.1

best_network = genetic_algorithm(data, input_size, layer_sizes, population_size, generations, mutation_rate)


In [36]:
correct = 0
for inputs, output in data:
    prediction = best_network.predict(inputs)
    prediction = np.round(prediction)
    if prediction == output:
        correct += 1

print("Accuracy: ",correct/len(data))

Accuracy:  0.78
Accuracy:  0.565
